# Скрипт подготовки обучающей, проверочной и тестовой выборок
Разворачивает в среде Colab наборы обучающей, проверочной и тестовой выборок (данных и меток) с пользовательскими параметрами на основе всей имеющейся разметки.

In [ ]:
# Скачиваем датасеты и аннотации по ссылкам с Google-диска
!gdown 1YSV*************************ZsxI
!gdown 1CsT*************************_rbd
!gdown 18gN*************************K_8Y

In [ ]:
# Распаковываем все скачанные архивы, удаляем служебные файлы и папки
TEMP_PATH = "/content/temp"
!rm -fr {TEMP_PATH}
!mkdir -p {TEMP_PATH}
!unzip -qo All_annotations.zip -d {TEMP_PATH}/labels
!unzip -qo Correct_image.zip -d {TEMP_PATH}/data
!unzip -qo correct_airport.zip -d {TEMP_PATH}/data
!rm -fr {TEMP_PATH}/data/__MACOSX
!find $TEMP_PATH -type f -name '.*' -exec rm -f {} +

In [ ]:
# Сливаем все метки в одну папку
!mkdir -p /content/temp/labels/all
!cp /content/temp/labels/base_dataset/* /content/temp/labels/all
!cp /content/temp/labels/airport_dataset/* /content/temp/labels/all
!echo "Всего меток: $(find /content/temp/labels/all -maxdepth 1 -type f | wc -l)"

In [ ]:
# Сливаем все изображения в одну папку
!mkdir -p /content/temp/data/all
!find /content/temp/data/Correct_image -type f -exec cp {} /content/temp/data/all \;
!find /content/temp/data/correct_airport -type f -exec cp {} /content/temp/data/all \;
!echo "Всего изображений: $(find /content/temp/data/all -maxdepth 1 -type f | wc -l)"

In [ ]:
# Создаем структуру папок и меток датасета
# Если вы хотите пересоздать датасет и/или изменить соотношение выборок - запустите сначала эту ячейку!
DATASET_PATH = "/content/dataset"
!rm -fr {DATASET_PATH}
!mkdir -p {DATASET_PATH}/data/train {DATASET_PATH}/data/test {DATASET_PATH}/data/val
!mkdir -p {DATASET_PATH}/labels/train {DATASET_PATH}/labels/test {DATASET_PATH}/labels/val

In [ ]:
# Скрипт подготавливает обучающую, проверочную и тестовую выборки для всех существующих меток/аннотаций
import random
import shutil
import os

# Пользовательские параметры, задайте нужные значения
VAL_SIZE = 0.15             # Размер проверочной выборки, доля от размера датасета
TEST_SIZE = 0.1             # Размер тестовой выборки, доля от размера датасета
RANDOM_STATE = 42           # Значение для повторения случайной выборки

SRC_DATA_PATH, SRC_LABELS_PATH = '/content/temp/data/all', '/content/temp/labels/all'
OUT_DATA_PATH, OUT_LABELS_PATH = '/content/dataset/data', '/content/dataset/labels'

# Получаем список всех меток (без расширения файла)
all_labels = sorted([file[:-4] for file in os.listdir(SRC_LABELS_PATH)])

# Размеры выборок в абсолютных числах
val_size_qty = int(len(all_labels) * VAL_SIZE)
test_size_qty = int(len(all_labels) * TEST_SIZE)

# Устанавливаем seed для генерации повторяющейся случайной выборки
random.seed(RANDOM_STATE)

# Создаем случайную выборку валидационных данных и копируем пары данные/метки в выходные папки
for _ in range(val_size_qty):
  randlabel = all_labels.pop(random.randint(0, len(all_labels) - 1))
  shutil.copy(f'{SRC_LABELS_PATH}/{randlabel}.txt', f'{OUT_LABELS_PATH}/val')
  shutil.copy(f'{SRC_DATA_PATH}/{randlabel}.tif', f'{OUT_DATA_PATH}/val')

# Создаем случайную выборку тестовых данных и копируем пары данные/метки в выходные папки
for _ in range(test_size_qty):
  randlabel = all_labels.pop(random.randint(0, len(all_labels) - 1))
  shutil.copy(f'{SRC_LABELS_PATH}/{randlabel}.txt', f'{OUT_LABELS_PATH}/test')
  shutil.copy(f'{SRC_DATA_PATH}/{randlabel}.tif', f'{OUT_DATA_PATH}/test')

# В списке осталась обучающая выборка, копируем пары данные/метки в выходные папки
for label in all_labels:
  shutil.copy(f'{SRC_LABELS_PATH}/{label}.txt', f'{OUT_LABELS_PATH}/train')
  shutil.copy(f'{SRC_DATA_PATH}/{label}.tif', f'{OUT_DATA_PATH}/train')


# Вывод проверочных значений
print(f'Размеры обучающей/проверочной/тестовой выборок: {len(all_labels)} / {val_size_qty} / {test_size_qty}')
print('Общий размер датасета:', len(all_labels) + val_size_qty + test_size_qty, '\n')

# Чтение файлов в папках и сверка соответствия изображений и меток
for folder, name in {'train':'обучающей', 'val':'проверочной', 'test':'тестовой'}.items():
  data = sorted([file[:-4] for file in os.listdir(f'{OUT_DATA_PATH}/{folder}')])
  labels = sorted([file[:-4] for file in os.listdir(f'{OUT_LABELS_PATH}/{folder}')])
  print(f'Данные и метки {name} выборки', 'совпали!' if data == labels else 'РАЗЛИЧАЮТСЯ, ОШИБКА!')